## New analysis goal: fixed skill sets from tourneys

Use df_matches to identify the group ids that use competitive rulesets.
This gives us the team ids we need.
Then we can get the team roster using https://fumbbl.com/api/team/get/1102662
we need the player ids here.

and the chosen skills using https://fumbbl.com/api/team/getOptions/1102662
this is a string of player ids, with skill ids.

through https://fumbbl.com/api/skill/list we can get a list of skills.
https://fumbbl.com/api/skill/list/2020 and this is the 2020 list.

World cup training: 9941 (dec 2020), 2 matches in dec 2020 using bb2016, then in okt 2021 for real. 429 matches.
SUper league: 15615
templars road to WC: 11605
entrainment tournois: 12879
NAF online tournaments : 9298
Tacklezone: 12013
Doppelbock: 13198
Eurobowl practice league: 15643
Eurobowl 2020 training: 12087 (eurobowl warsaw)

In [ ]:
import pandas as pd
import numpy as np
import plotnine as p9
import requests

from mizani.formatters import date_format

# point this to the location of the HDF5 datasets
path_to_datasets = 'datasets/current/'

# FUMBBL matches
target = 'df_matches.h5'
df_matches = pd.read_hdf(path_to_datasets + target) 



In [ ]:
# FUMBBL inducements
target = 'inducements.h5'
inducements = pd.read_hdf(path_to_datasets + target) 

In [ ]:
df_matches.query('match_id == 4257923')['team2_id']

In [ ]:
inducements.query('match_id == 4335359')

In [ ]:
#fetch skill code list
%run write_json_file.py

In [ ]:
api_string = "https://fumbbl.com/api/skill/list/2020"

skill_list = requests.get(api_string)
# PM here save JSON as file
skill_list = skill_list.json()

write_json_file(skill_list, fname = "skill_list_bb2020.json")

In [ ]:
team_id = 1050197 # test malta necros

api_string = "https://fumbbl.com/api/team/get/" + str(team_id)

team = requests.get(api_string)
# PM here save JSON as file
team = team.json()

write_json_file(team, fname = str(team_id) + ".json")

api_string = "https://fumbbl.com/api/team/getOptions/" + str(team_id)

r = requests.get(api_string)
r = r.json()
# the following row is required to dodge this bug #203:
r['tournamentSkills'] = json.loads(r['tournamentSkills'])

write_json_file(r, fname = str(team_id) + "_skills.json")

https://fumbbl.com/api/ruleset/get/188
contains info about gold and skills.

From the roster id we can infer the bb version.

In [ ]:
df_matches.query('league == 9941 and match_date > "2021-09-20"')
#df_matches.query('league == 9941')['team2_id']

In [ ]:
team_id = 1081222
